## Information Retrieval

This is the document retrieval and sentence retrieval part of the project.

###  1. Unpack the zip file.
    
Unpack the 'wiki-pages-text.zip' in the current directory.

In [5]:
import zipfile
def unpack():
    with zipfile.ZipFile('wiki-pages-text.zip') as file:
        file.extractall()
unpack()

### 2. Load file. 

Load the training dataset and the wiki txt file.

In [7]:
import os
import json
import pandas as pd

with open('train.json', 'r') as f:  # load training dataset
        train_data = json.load(f)   
print("Length of the train data is: " + str(len(train_data)))

with open('devset.json', 'r') as f1:  # load dev dataset
        dev_data = json.load(f1) 
print("Length of the dev data is: " + str(len(dev_data)))

with open('devset_result.json', 'r') as f2:  # store result 
        res_data = json.load(f2) 
print("Length of the dev result data is: " + str(len(res_data)))

with open('test-unlabelled.json', 'r') as f3:  # store result 
     test_data = json.load(f3) 
print("Length of the test data is: " + str(len(test_data)))
        
# appeand all the wiki txt sentences to one document
def loadfile(folder): 
    corpus = []
    list_of_files = os.listdir(folder)
    
    for file in list_of_files:
        try:
            filename = os.path.join(folder, file)
            with open(filename, 'r') as doc:
                for line in doc:
                    corpus.append(line)     
        except Exception as e:
            print("No files found here!")
            raise e
    return corpus
corpus = loadfile("wiki-pages-text")
print("Length of the corpus is: " + str(len(corpus)))

corpus_df = pd.DataFrame(corpus)
print(corpus_df.shape)

Length of the train data is: 145449
Length of the dev data is: 5001
Length of the dev result data is: 5001
Length of the test data is: 14997
Length of the corpus is: 25248397
(25248397, 1)


### 3. Preprocess 

Preprocess includes: strip punctuations, tokenize,lemma, lower case, remove stop words.

In [29]:
import nltk
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
lemmatizer = nltk.stem.WordNetLemmatizer()

nltk.download('wordnet')
def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma

def pre_process(comment) -> str:
    # lower cased
    comment = comment.lower()
    # tokenize
    words = tokenizer.tokenize(comment)
    # lemmatize 
    words = [lemmatize(w) for w in words]
    processed_comment = " ".join(words)
    return processed_comment

def process_dataset(dataset: pd.DataFrame) -> pd.DataFrame:
    processed_corpus = dataset['wiki_text'].apply(lambda text: pre_process(text))
    return processed_corpus

corpus_df.columns = ['wiki_text']
print(corpus_df.loc[:10])

processed_corpus_df = process_dataset(corpus_df.loc[:2000])
processed_corpus_df.to_pickle("./processed_corpus.pkl")
print(processed_corpus_df.loc[:10])

                                            wiki_text
0   Alexander_McNair 0 Alexander McNair -LRB- May ...
1   Alexander_McNair 1 He was the first Governor o...
2   Alexander_McNair 4 McNair was born in Lancaste...
3   Alexander_McNair 5 His grandfather , David McN...
4   Alexander_McNair 6 David McNair , Jr. , Alexan...
5   Alexander_McNair 7 Alexander went to school as...
6   Alexander_McNair 8 He reached an agreement wit...
7       Alexander_McNair 9 Alexander was defeated .\n
8   Alexander_McNair 10 He became a member of the ...
9   Alexander_McNair 13 In 1804 , McNair traveled ...
10  Alexander_McNair 14 In that year he married Ma...
0     alexander_mcnair 0 alexander mcnair lrb may 5 ...
1     alexander_mcnair 1 he be the first governor of...
2     alexander_mcnair 4 mcnair be bear in lancaster...
3     alexander_mcnair 5 his grandfather david mcnai...
4     alexander_mcnair 6 david mcnair jr alexander s...
5     alexander_mcnair 7 alexander go to school a a ...
6     alexander_

#### Load processed dataframe.

In [30]:
load_processed_corpus_df = pd.read_pickle("./processed_corpus.pkl")
print(load_processed_corpus_df.shape)
print(load_processed_corpus_df.loc[:10])

(2001,)
0     alexander_mcnair 0 alexander mcnair lrb may 5 ...
1     alexander_mcnair 1 he be the first governor of...
2     alexander_mcnair 4 mcnair be bear in lancaster...
3     alexander_mcnair 5 his grandfather david mcnai...
4     alexander_mcnair 6 david mcnair jr alexander s...
5     alexander_mcnair 7 alexander go to school a a ...
6     alexander_mcnair 8 he reach an agreement with ...
7                alexander_mcnair 9 alexander be defeat
8     alexander_mcnair 10 he become a member of the ...
9     alexander_mcnair 13 in 1804 mcnair travel to w...
10    alexander_mcnair 14 in that year he marry marg...
Name: wiki_text, dtype: object


### 4. Use Sklearn to build tf-idf.

tfidf_vectorizer is the tf-idf model.

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

max_features = 20000
tfidf_vectorizer = TfidfVectorizer(max_features=max_features, norm='l2')
tfidf_vectorizer.fit(load_processed_corpus_df)

pickle.dump(tfidf_vectorizer, open("tfidf_vectorizer.pickle","wb"))

# load_tfidf_vectorizer = pickle.load(open("tfidf_vectorizer.pickle","rb"))

####  Test tfidf

In [49]:
query = "Alexander  Alatskivi"
pre_query= pre_process(query)
print(pre_query)
query_df = pd.DataFrame([pre_query])
print(query_df[0])
res = tfidf_vectorizer.transform(query_df[0])
print(res)

feature_names = tfidf_vectorizer.get_feature_names()
for col in res.nonzero()[1]:
    print(feature_names[col], ' - ', res[0, col])

alexander alatskivi
0    alexander alatskivi
Name: 0, dtype: object
  (0, 911)	0.5919900134396463
  (0, 791)	0.8059452983842806
alexander  -  0.5919900134396463
alatskivi  -  0.8059452983842806


### Use page identifier to reduce the search space.

Build a dictionary that consists the <b>page identifier</b> as the key and the document index(ranges form 0 to 25248397 ） as value.

Examples looks like: "Alexander McNair" : [1,2,3,4...18] 

<b>To do:</b> the keywords list may also consider synonyms for each keyword.


In [36]:
final_dic = {}

for id, doc in enumerate(corpus):
    final_dic.setdefault((doc.split()[0].replace("_", " ")), []).append(id)

keys = list(final_dic.keys())
print("Length of keys = {}".format(len(keys)))

Length of keys = 5396106


###  Build a dictionary for all the unique words in the keys.

Keywords is a dictionary that stores all the unique page identifier word and its corresponing document index list.

In [71]:

def create_keywords(keys, final_dic):
    keywords = {}
    for key in keys:
        words = key.split()
        for word in words:
            print(word)
            keywords[word] = final_dic[key]
    return keywords

# keywords should then remove stop words
print(keys[:20])
keywords = create_keywords(keys[:20], final_dic)
#print("Length of keywords = {}".format(len(keywords)))
#print(keywords)
#print('\n')
#print(keywords.get('Alexander'))

['Alexander McNair', 'Alatskivi', 'An American Girl Story - Maryellen 1955-COLON- Extraordinary Christmas', 'Alta Outcome Document', 'Al ‘Urban', 'Albano buoy system', 'Amuka', 'Aleksandr Abdulkhalikov', 'Amalia Ciardi Duprè', 'Akbil', 'All These Years', 'Ambulance services of Victoria', 'Alabama elections, 2018', 'Akarsu, Ardanuç', 'Albertinovac', 'All-India Yadav Mahasabha', 'Alejandro Rodríguez López', 'Aleksandr Luzin', 'American Thighs', 'Alan Brown -LRB-Australian politician-RRB-']
Alexander
McNair
Alatskivi
An
American
Girl
Story
-
Maryellen
1955-COLON-
Extraordinary
Christmas
Alta
Outcome
Document
Al
‘Urban
Albano
buoy
system
Amuka
Aleksandr
Abdulkhalikov
Amalia
Ciardi
Duprè
Akbil
All
These
Years
Ambulance
services
of
Victoria
Alabama
elections,
2018
Akarsu,
Ardanuç
Albertinovac
All-India
Yadav
Mahasabha
Alejandro
Rodríguez
López
Aleksandr
Luzin
American
Thighs
Alan
Brown
-LRB-Australian
politician-RRB-


In [67]:
import numpy as np
def calculate_cosines(claim_tfidf, evi_tfidf) -> np.ndarray:
    cosines = np.zeros((claim_tfidf.shape[0], 1))
    for i in range(len(cosines)):
        claim_vector = claim_tfidf[i]
        evi_vector = evi_tfidf[i]
        cosine_matrix = cosine_similarity([claim_vector.toarray()[0], evi_vector.toarray()[0]])
        cosines[i][0] = cosine_matrix[0][1]
    return cosines

print(keywords.get('Alexander'))
# testQuery = "Alexander Alatskivi"
# query_tfidf = tfidf_vectorizer.transform(testQuery)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



###  Retrieval Evidence
Given a query, tokenize it first, then for each token in the query, find it in the keywords dictionary. 


To do: smater select the in the range of its alphabet.

In [64]:
# Given a query, returns a list that contains all the document index values.
def extract_sentences(query, keys, final_dic):
    retrievaled_sentences = []
    for word in  nltk.tokenize.word_tokenize(query):
        retrievaled_sentences.append([keywords.get(word)])
            
    retrievaled_sentences = [item for sublist in retrievaled_sentences for item in sublist]
    return retrievaled_sentences

testQuery = "Alexander Alatskivi"
print(extract_sentences(testQuery,keys[:100],final_dic))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



If there is a match or substring match, then retrieve that sentence by using index value to find the raw sentence in txt file. 

Find all the related sentences by this way and then builds a inverted index and 
uses BM25 to rank and to retrieval top K sentences as the evidence.


In [14]:
# Use the result from above step, find all the raw sentences in txt file.
# return a list of the doc id orderd by rnaking tfidf or bm25 score.

def retrieval_evidence(query, keys, final_dic):
    processed_doc = [] # processed_docs stores the list of processed docs
    vocab = {}
    unique_id = 0
    rank_result = []
    
    retrievaled_sentences = extract_sentences(query,keys,final_dic)
    
    # find the row sentences and save them in processed_doc
    for retrievaled_sentence in retrievaled_sentences:
        norm_sentence = preprocess(document[retrievaled_sentence])
        for token in norm_sentence:
            if token not in vocab:
                vocab.update({token: unique_id})     
                unique_id = unique_id + 1
        processed_doc.append(norm_sentence) 
    
    # calculate doc term freqs and build an inverted index
    doc_term_freqs = doc_term_freq(processed_doc)
    invindex = InvertedIndex(vocab, doc_term_freqs)
    
    processed_query = preprocess(query)
    bm25_results = BM25(processed_query, invindex, vocab)
    tfidf_results = tfidf(processed_query, invindex, vocab)
    
    for rank, res in enumerate(tfidf_results):
        # print("RANK {:2d} DOCID {:8d} SCORE {:.3f} CONTENT {:}".format(rank+1,res[0],res[1],document[res[0]]))
        rank_result.append((res[0]))
    return rank_result

# test_query = "Nikolaj Coster-Waldau worked with the Fox Broadcasting Company."
# test_query = "Alexander Alatskivi"
# print(retrieval_evidence(test_query,keys[:100],final_dic))

### Write the result to json file.

The dataset used is 'devset.json', the predicted result is 'devset_result.json'.


In [18]:
def get_evidence(res_data):
    for key in list(res_data)[:10]:
        res_data[key]["evidence"] = []
        tfidf_result = retrieval_evidence(res_data[key]["claim"],keys[:5000],final_dic)
        for res in tfidf_result:
            res_data[key]["evidence"].append([document[res].split()[0], document[res].split()[1]])
    return res_data

#testing, for top 10 instances in the dev, and only consider top 100 sentences in the documents.
predicted_train = get_evidence(res_data)

#for key in list(predicted_train)[:10]:
 #   print(predicted_train[key])
  #  print("\n")

### Gensim library.

Use gensim library to calculate the coscine similarity in IR.

In [30]:
import gensim

raw_documents = ["I'm taking the show on the road.",
                 "My socks are a force multiplier.",
                 "I am the barber who cuts everyone's hair who doesn't cut their own.",
                 "Legend has it that the mind is a mad monkey.",
                 "I make my own fun."]

from nltk.tokenize import word_tokenize
gen_docs = [[w.lower() for w in word_tokenize(text)] for text in raw_documents]

dictionary = gensim.corpora.Dictionary(gen_docs)
print(gen_docs)
print(dictionary)
print(dictionary.token2id['socks'])
print("Number of words in dictionary:",len(dictionary))
for i in range(len(dictionary)):
    print(i, dictionary[i])

corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]
print(corpus)

tf_idf = gensim.models.TfidfModel(corpus)
print(tf_idf)












[['i', "'m", 'taking', 'the', 'show', 'on', 'the', 'road', '.'], ['my', 'socks', 'are', 'a', 'force', 'multiplier', '.'], ['i', 'am', 'the', 'barber', 'who', 'cuts', 'everyone', "'s", 'hair', 'who', 'does', "n't", 'cut', 'their', 'own', '.'], ['legend', 'has', 'it', 'that', 'the', 'mind', 'is', 'a', 'mad', 'monkey', '.'], ['i', 'make', 'my', 'own', 'fun', '.']]
Dictionary(36 unique tokens: ["'m", '.', 'i', 'on', 'road']...)
13
Number of words in dictionary: 36
0 'm
1 .
2 i
3 on
4 road
5 show
6 taking
7 the
8 a
9 are
10 force
11 multiplier
12 my
13 socks
14 's
15 am
16 barber
17 cut
18 cuts
19 does
20 everyone
21 hair
22 n't
23 own
24 their
25 who
26 has
27 is
28 it
29 legend
30 mad
31 mind
32 monkey
33 that
34 fun
35 make
[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2)], [(1, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1)], [(1, 1), (2, 1), (7, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 2)], [(1, 